In [1]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from tensorflow.examples .tutorials.mnist import input_data

tf.set_random_seed(777)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.001
training_epochs = 20
batch_size = 100

In [4]:
class Model :    
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            # 데이터 셋팅 
            self.training = tf.placeholder(tf.bool)
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            # Layer 1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3],
                                    padding='SAME', activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2],
                                           padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1, rate=0.3, training=self.training)
            # Layer 2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3],
                                    padding='SAME', activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2],
                                           padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=self.training)
            # Layer 3
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3,3],
                                    padding='SAME', activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2],
                                           padding="SAME", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3, rate=0.3, training=self.training)        
            
            flat = tf.reshape(dropout3, [-1, 128 * 4* 4])
            dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
            
            dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=self.training)
            
            self.logits = tf.layers.dense(inputs=dropout4, units=10)
            
            
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                        logits=self.logits, labels=self.Y))
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
    
    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict={self.X:x_test, self.training:training})
    
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test,self.Y: y_test, self.training: training})
    
#     def get_accuracy(self, x_test, y_test, training=False):
#         return self.sess.run(self.accuracy, feed_dict={self.X:x_test, self.Y:y_test ,self.training:training} )
    
    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X:x_data, self.Y:y_data ,self.training:training} )
    

In [5]:
sess = tf.Session()

models=[]
num_models = 2

for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
print('Learning Started!')

for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c /total_batch
            
            
    print('Epoch:', '%04d' % (epoch +1), 'cost= ', avg_cost_list)
    
    
print('Learning Finished')

Learning Started!
Epoch: 0001 cost=  [0.29603327 0.28955353]
Epoch: 0002 cost=  [0.09013931 0.0884696 ]
Epoch: 0003 cost=  [0.06779923 0.06582272]
Epoch: 0004 cost=  [0.05740262 0.05660918]
Epoch: 0005 cost=  [0.04862272 0.04811735]
Epoch: 0006 cost=  [0.04471094 0.04552414]
Epoch: 0007 cost=  [0.0407184  0.04033074]
Epoch: 0008 cost=  [0.03897906 0.03799791]
Epoch: 0009 cost=  [0.03376748 0.03548769]
Epoch: 0010 cost=  [0.03379376 0.03334266]
Epoch: 0011 cost=  [0.03105786 0.03204114]
Epoch: 0012 cost=  [0.03113749 0.03061803]
Epoch: 0013 cost=  [0.03010297 0.03000734]
Epoch: 0014 cost=  [0.02763277 0.02641703]
Epoch: 0015 cost=  [0.02717784 0.02590453]
Epoch: 0016 cost=  [0.02578294 0.02623076]
Epoch: 0017 cost=  [0.02492986 0.02638959]
Epoch: 0018 cost=  [0.02470051 0.02245654]
Epoch: 0019 cost=  [0.0227764  0.02553414]
Epoch: 0020 cost=  [0.02339529 0.02456254]
Learning Finished


In [7]:
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])

for m_idx, m in enumerate(models):
    print(m_idx,'???' ,m)
    print(m_idx, 'Accuracy:', m.get_accuracy(mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels,1))

ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

0 ??? <__main__.Model object at 0x00000240DED79FC8>


AttributeError: 'Model' object has no attribute 'accuracy'